<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-16 15:42:12
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.42 C
-------------------
Today PnL: 12.31 K (0.09%)
Current PnL: -32.36 L (-20.23%)
CY Booked + Current PnL: -16.68 L (-10.43%)
-------------------
Total profit:  1.22 L
Total loss:  -33.58 L
-------------------
Total Booked + Current PnL: 10.30 L (7.74%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.04%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 97.73 L (68.78%)
Deployed:  1.33 C
Current:  1.42 C
CAGR/XIRR %: 3.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.26,5.13,15.64,21.57,16110.0,5025.0,103005.0,397.04,-9.21,64.0,H-SC,3.85,167.0,0.31,0.75,50.32,XR,ATH,FINANCE
18,COALINDIA,3.31,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.37,185.0,0.25,1.14,23.94,XY25,ATH,MINING
77,TTKPRESTIG,-1.00,-27.44,37.94,0.09,27743.0,-27653.0,73124.0,770.00,69.69,39.0,M-SC,8.94,250.0,-1.00,0.53,0.64,OX40N,NTT,DURABLES
36,ICICIGI,0.31,1.81,17.22,19.34,35083.0,3623.0,203732.0,2252.93,-14.44,58.0,X-MC,3.14,66.0,0.10,1.49,18.04,X40,ATH,INSURANCE
51,MEDANTA,0.53,-4.39,29.77,24.08,36135.0,-5569.0,121381.0,1486.00,-5.04,53.0,X-SC,11.64,87.0,-0.15,0.89,11.17,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.82,-56.29,233.33,45.70,180324.0,-99532.0,77283.0,26.40,-19.77,70.0,M-SC,26.02,204.0,-0.55,0.56,27.33,XY24,NTT,TRAVEL
30,HAPPSTMNDS,3.76,-51.82,283.70,84.85,175168.0,-66421.0,61744.0,1480.71,-42.72,37.0,H-SC,25.42,152.0,-0.38,0.45,3.76,AR,ATH,IT
18,COALINDIA,3.31,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.37,185.0,0.25,1.14,23.94,XY25,ATH,MINING
65,SIEMENS,3.18,-13.60,45.28,25.51,72801.0,-25315.0,160780.0,4671.50,35.03,59.0,H-LC,2.28,51.0,-0.35,1.17,18.48,AR,ATH,ELECTRICAL
23,DIXON,2.93,-18.53,61.11,31.26,114897.0,-42752.0,188016.0,18931.72,-63.60,55.0,X-MC,26.39,54.0,-0.37,1.37,15.39,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,-7.96,-48.84,240.40,74.15,201217.0,-79904.0,83701.0,838.0,-250.34,42.0,H-SC,24.56,145.0,-0.40,0.61,7.29,XR,NTT,CHEMICALS
69,SURYODAY,-5.04,-29.76,75.15,23.03,94528.0,-53285.0,125786.0,216.0,38.16,28.0,H-SC,2.15,166.0,-0.56,0.92,24.81,XR,NTT,BANKS
55,RAJESHEXPO,-5.00,-67.35,206.45,0.07,92475.0,-92384.0,44793.0,518.0,1541.41,45.0,L-SC,8.96,268.0,-1.00,0.33,11.40,OX40N,NTT,JEWELLERY
5,ANGELONE,-4.65,5.36,17.89,24.20,57534.0,16350.0,321600.0,3033.0,52.88,48.0,X-SC,3.64,97.0,0.28,2.35,34.69,X40N,NTT,FINANCE
66,SIS,-4.51,-28.21,68.91,21.26,54930.0,-31319.0,79713.0,528.0,1875.13,37.0,H-SC,4.11,168.0,-0.57,0.58,9.89,OX40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.78,1.89,104.06,107.91,156540.0,2788.0,150432.0,1641.55,-8.17,62.0,M-SC,9.83,220.0,0.02,1.10,13.18,OX40N,ATH,CABLES
77,TTKPRESTIG,-1.00,-27.44,37.94,0.09,27743.0,-27653.0,73124.0,770.00,69.69,39.0,M-SC,8.94,250.0,-1.00,0.53,0.64,OX40N,NTT,DURABLES
66,SIS,-4.51,-28.21,68.91,21.26,54930.0,-31319.0,79713.0,528.00,1875.13,37.0,H-SC,4.11,168.0,-0.57,0.58,9.89,OX40N,NTT,MISC
40,INDIGOPNTS,1.47,-29.86,42.60,0.02,52159.0,-52121.0,122438.0,1408.00,86.94,32.0,M-SC,7.99,222.0,-1.00,0.89,6.50,OX40N,NTT,PAINTS
72,TATAELXSI,-0.06,-30.52,90.02,32.03,82457.0,-40236.0,91599.0,9161.00,-13.14,32.0,H-SC,6.37,158.0,-0.49,0.67,2.60,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,3.31,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.37,185.0,0.25,1.14,23.94,XY25,ATH,MINING
26,FINCABLES,-0.78,1.89,104.06,107.91,156540.0,2788.0,150432.0,1641.55,-8.17,62.0,M-SC,9.83,220.0,0.02,1.10,13.18,OX40N,ATH,CABLES
50,MASFIN,2.26,5.13,15.64,21.57,16110.0,5025.0,103005.0,397.04,-9.21,64.0,H-SC,3.85,167.0,0.31,0.75,50.32,XR,ATH,FINANCE
1,ABB,1.86,8.03,34.75,45.57,98212.0,21005.0,282624.0,7934.00,-30.61,71.0,H-MC,2.88,125.0,0.21,2.06,25.50,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,2.26,5.13,15.64,21.57,16110.0,5025.0,103005.0,397.04,-9.21,64.0,H-SC,3.85,167.0,0.31,0.75,50.32,XR,ATH,FINANCE
1,ABB,1.86,8.03,34.75,45.57,98212.0,21005.0,282624.0,7934.00,-30.61,71.0,H-MC,2.88,125.0,0.21,2.06,25.50,AR,NTT,ELECTRICAL
26,FINCABLES,-0.78,1.89,104.06,107.91,156540.0,2788.0,150432.0,1641.55,-8.17,62.0,M-SC,9.83,220.0,0.02,1.10,13.18,OX40N,ATH,CABLES
39,INDIAMART,0.26,-5.21,118.09,106.72,138055.0,-6429.0,116907.0,4810.62,-57.23,48.0,H-SC,7.65,140.0,-0.05,0.85,16.95,AR,ATH,MISC
85,ZYDUSLIFE,0.64,-3.99,41.27,35.63,83039.0,-8365.0,201209.0,1286.17,-16.22,53.0,H-MC,5.74,120.0,-0.10,1.47,12.02,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-0.26,-7.04,44.41,34.24,129177.0,-22015.0,290873.0,1972.00,-26.71,19.0,X-LC,4.48,3.0,-0.17,2.12,0.66,X40,NTT,IT
33,HCLTECH,0.45,-5.13,30.54,23.83,70090.0,-12421.0,229503.0,1908.19,-0.33,24.0,X-LC,4.99,8.0,-0.18,1.67,10.11,X40,ATH,IT
73,TCS,0.53,-25.58,59.30,18.55,166921.0,-96751.0,281486.0,4311.59,-32.81,26.0,X-LC,3.06,2.0,-0.58,2.05,0.53,X40,ATH,IT
13,BERGEPAINT,0.09,-17.62,47.06,21.14,88130.0,-40067.0,187272.0,680.00,-19.34,30.0,X-MC,11.57,68.0,-0.45,1.37,0.85,XY24,NTT,PAINTS
8,AWL,1.16,-33.91,137.86,57.19,297119.0,-110605.0,215522.0,485.00,-62.20,31.0,X-SC,17.26,81.0,-0.37,1.57,1.16,XY24,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.35,0.54,42.93,43.70,107072.0,1343.0,249411.0,735.00,-8.76,52.0,X-MC,1.80,70.0,0.01,1.82,15.67,XY24,BTT,FMCG
65,SIEMENS,3.18,-13.60,45.28,25.51,72801.0,-25315.0,160780.0,4671.50,35.03,59.0,H-LC,2.28,51.0,-0.35,1.17,18.48,AR,ATH,ELECTRICAL
78,UNITDSPR,-0.10,2.92,17.34,20.77,41785.0,6828.0,240972.0,1644.00,-8.78,58.0,X-MC,2.38,64.0,0.16,1.76,10.53,X40N,NTT,BREWERIES
81,VBL,1.46,-7.92,47.34,35.67,137680.0,-25010.0,290832.0,671.64,-18.65,43.0,X-LC,2.82,13.0,-0.18,2.12,4.84,X40N,ATH,FMCG
34,HINDUNILVR,0.60,-9.79,26.00,13.67,60294.0,-25163.0,231900.0,2922.00,-33.13,42.0,X-LC,2.84,9.0,-0.42,1.69,9.48,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
2,ABBOTINDIA,-0.64,-11.81,33.87,18.07,53427.0,-21115.0,157740.0,35195.00,-24.86,34.0,X-MC,10.32,58.0,-0.40,1.15,0.00,X40,ATH,PHARMA
3,ACC,-0.30,-31.41,139.31,64.15,227382.0,-74731.0,163220.0,3906.00,-48.05,37.0,X-SC,8.21,85.0,-0.33,1.19,0.00,XY24,BTT,CEMENT
12,BATAINDIA,-2.75,-46.53,152.88,35.20,105174.0,-59875.0,68795.0,2096.00,-9.75,34.0,X-SC,21.24,91.0,-0.57,0.50,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.39,-54.05,231.45,52.31,154382.0,-78458.0,66702.0,1176.00,-51.12,33.0,X-SC,14.22,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
53,PGHH,-1.06,-13.30,52.26,32.01,96744.0,-28400.0,185120.0,17616.87,-41.36,34.0,X-MC,9.26,57.0,-0.29,1.35,0.06,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.94,-30.25,104.78,42.84,47511.0,-19662.0,45344.0,424.00,-54.27,51.0,X-SC,19.57,80.0,-0.41,0.33,9.07,XY24,NTT,MISC
58,RELAXO,-1.39,-54.05,231.45,52.31,154382.0,-78458.0,66702.0,1176.00,-51.12,33.0,X-SC,14.22,92.0,-0.51,0.49,0.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-2.75,-46.53,152.88,35.20,105174.0,-59875.0,68795.0,2096.00,-9.75,34.0,X-SC,21.24,91.0,-0.57,0.50,0.00,X40,NTT,FOOTWEAR
51,MEDANTA,0.53,-4.39,29.77,24.08,36135.0,-5569.0,121381.0,1486.00,-5.04,53.0,X-SC,11.64,87.0,-0.15,0.89,11.17,XY24,NTT,HEALTHCARE
35,HONAUT,-0.57,-23.39,87.07,43.32,108646.0,-38092.0,124780.0,58357.33,-32.74,37.0,X-SC,10.63,90.0,-0.35,0.91,1.32,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,1.34,-22.07,42.16,10.78,78954.0,-53045.0,187273.0,452.00,-53.90,42.0,X-LC,19.01,1.0,-0.67,1.37,3.15,X40,NTT,FMCG
73,TCS,0.53,-25.58,59.30,18.55,166921.0,-96751.0,281486.0,4311.59,-32.81,26.0,X-LC,3.06,2.0,-0.58,2.05,0.53,X40,ATH,IT
42,INFY,-0.26,-7.04,44.41,34.24,129177.0,-22015.0,290873.0,1972.00,-26.71,19.0,X-LC,4.48,3.0,-0.17,2.12,0.66,X40,NTT,IT
75,TMPV,-0.79,-22.68,59.05,22.98,103141.0,-51231.0,174667.0,600.00,-82.53,60.0,X-LC,3.25,4.0,-0.50,1.27,11.66,XY24,NTT,AUTO
33,HCLTECH,0.45,-5.13,30.54,23.83,70090.0,-12421.0,229503.0,1908.19,-0.33,24.0,X-LC,4.99,8.0,-0.18,1.67,10.11,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.63,-31.20,93.15,32.89,49479.0,-24088.0,53118.0,1800.00,-350.00,60.0,L-MC,11.06,258.0,-0.49,0.39,46.28,XR,NTT,BANKS
67,SONACOMS,-0.54,-9.16,53.12,39.09,48813.0,-9267.0,91892.0,804.02,-28.03,61.0,M-MC,5.13,193.0,-0.19,0.67,30.00,AR,ATH,AUTO
50,MASFIN,2.26,5.13,15.64,21.57,16110.0,5025.0,103005.0,397.04,-9.21,64.0,H-SC,3.85,167.0,0.31,0.75,50.32,XR,ATH,FINANCE
79,VAIBHAVGBL,-2.80,-24.05,111.67,60.77,155024.0,-43952.0,138823.0,521.00,60.33,56.0,H-SC,0.81,160.0,-0.28,1.01,25.86,XR,NTT,JEWELLERY
18,COALINDIA,3.31,3.90,14.75,19.23,23058.0,5864.0,156325.0,484.83,24.82,50.0,L-LC,11.37,185.0,0.25,1.14,23.94,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,1.86,8.03,34.75,45.57,98212.0,21005.0,282624.0,7934.00,-30.61,71.0,H-MC,2.88,125.0,0.21,2.06,25.50,AR,NTT,ELECTRICAL
11,BANDHANBNK,1.50,-19.22,138.19,92.42,310963.0,-53533.0,225026.0,400.00,93.25,68.0,H-SC,7.83,174.0,-0.17,1.64,30.05,XY24,NTT,BANKS
79,VAIBHAVGBL,-2.80,-24.05,111.67,60.77,155024.0,-43952.0,138823.0,521.00,60.33,56.0,H-SC,0.81,160.0,-0.28,1.01,25.86,XR,NTT,JEWELLERY
7,ATULAUTO,-0.50,-14.13,69.53,45.57,117693.0,-27863.0,169269.0,844.00,3667.57,65.0,M-SC,5.14,248.0,-0.24,1.23,28.10,XY24,NTT,AUTO
67,SONACOMS,-0.54,-9.16,53.12,39.09,48813.0,-9267.0,91892.0,804.02,-28.03,61.0,M-MC,5.13,193.0,-0.19,0.67,30.00,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.25
1,25,44.68
2,50,76.84


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.21
MC    30.47
LC    23.30
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.71
X40      23.16
X40N     15.26
AR        9.05
XY25      8.82
XR        8.78
OX40N     7.33
SR        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.00
H-SC    22.67
X-LC    19.91
X-SC    11.86
M-SC    10.87
H-MC     4.99
M-MC     2.09
H-LC     1.17
L-LC     1.14
M-LC     1.08
L-SC     0.81
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.03,-14.98,53.62
FINANCE,13.40,-13.66,58.38
IT,11.56,-34.74,101.97
MISC,7.10,-33.29,87.81
ELECTRICAL,6.32,-8.31,47.84
PAINTS,5.14,-31.20,50.70
INSURANCE,4.88,-1.47,36.19
PHARMA,4.14,-3.70,36.34
AUTO,3.17,-20.27,61.87


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3308252.0,21
AR,1328982.0,10
X40,1321287.0,15
XR,1289645.0,12
X40N,1029384.0,10
OX40N,785187.0,10
XY25,410291.0,6
SR,299491.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3683108.0,24
M-SC,1424435.0,14
X-MC,1361286.0,15
X-SC,1178924.0,10
X-LC,1160329.0,12
H-MC,394952.0,3
L-SC,180772.0,2
M-LC,143681.0,1
M-MC,99694.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1254610.0      6
           AR         951652.0      5
           XR         828628.0      7
M-SC       XY24       826425.0      6
X-SC       XY24       608147.0      4
X-LC       X40        595119.0      6
X-MC       X40        546182.0      7
           X40N       410268.0      4
X-SC       X40N       390791.0      4
H-SC       OX40N      348727.0      4
M-SC       OX40N      343985.0      5
H-SC       SR         299491.0      2
X-LC       X40N       228325.0      2
H-MC       XY24       213701.0      1
X-LC       XY24       210167.0      2
X-MC       XY25       209634.0      2
           XY24       195202.0      2
H-MC       AR         181251.0      2
X-SC       X40        179986.0      2
M-SC       XR         179560.0      2
M-LC       XR         143681.0      1
X-LC       XY25       126718.0      2
L-SC       OX40N       92475.0      1
           XR          88297.0      1
M-SC       AR          74465.0      1
H-LC       AR          72801.0      1
M-MC       XY25        50881.0      1
L-MC       XR          49479.0      1
M-MC       AR          48813.0      1
L-LC       XY25        23058.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
